In [1]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.stats as stats
import netCDF4
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import sys
import random
import os
import xarray as xr
import numpy as np
import numpy.ma as ma
import pandas as pd
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as patches
import matplotlib.ticker as ticker
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable
from eofs.xarray import Eof
import time
import h5py
from scipy.signal import correlation_lags
from scipy.stats import pearsonr
from scipy.stats import linregress
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
os.chdir('/home/giantstep5/rjones98/machine_learning/input_datasets/ERA5')

ds_daily = xr.open_dataset('ds_fulldegree_3hr_2010_2023_v3.nc')
ds_daily = ds_daily.isel(lon=slice(0, 148), lat=slice(0, 104))

dataset_cnn = ds_daily

feature_name   = ['cape', 'precipitation', 'lsm', 'rh', 'shear', 't2m', 'wcd']
output_name    = ['ltg']

indices_train_val = np.arange(0, 35064)  #2010 to 2021
idx_test = np.arange(35064, 40904)       #2022 and 2023
idx_train, idx_val = train_test_split(indices_train_val, test_size=0.25, random_state=13) #Save 25% of training dataset for validation
zdim, _, ydim, xdim = ds_daily[feature_name].to_array().shape

train_dataset_cnn_X = dataset_cnn[feature_name].isel(time=idx_train)
val_dataset_cnn_X = dataset_cnn[feature_name].isel(time=idx_val)
test_dataset_cnn_X  = dataset_cnn[feature_name].isel(time=idx_test)
train_dataset_cnn_y = dataset_cnn[output_name].isel(time=idx_train)
val_dataset_cnn_y = dataset_cnn[output_name].isel(time=idx_val)
test_dataset_cnn_y  = dataset_cnn[output_name].isel(time=idx_test)

In [ ]:
seed = 13
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Define the PyTorch model
class SimpleCNN(nn.Module):
    def __init__(self, zdim, ydim, xdim):
        super(SimpleCNN, self).__init__()
        self.encoder0 = self.encoder_block(zdim, 32)
        self.encoder1 = self.encoder_block(32, 16)
        self.center = self.conv_block(16, 8)
        self.decoder1 = self.decoder_block(8, 16)
        self.decoder0 = self.decoder_block(16, 32)
        self.outputs = nn.Sequential(
            nn.Conv2d(32, 1, kernel_size=1, padding=0),
#            nn.ReLU()  # Ensures non-negative output
        )

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def encoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            self.conv_block(in_channels, out_channels),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def decoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        encoder0 = self.encoder0(x)
        encoder1 = self.encoder1(encoder0)
        center = self.center(encoder1)
        decoder1 = self.decoder1(center)
        decoder0 = self.decoder0(decoder1)
        outputs = self.outputs(decoder0)
        return outputs

# Initialize model, loss function, and optimizer
zdim, ydim, xdim = 7, 104, 148  # Adjust these based on your data
model = SimpleCNN(zdim, ydim, xdim).to('cpu')
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [ ]:
def normalize(x, m, s):
    return (x - m).groupby('time.month') / s

def unnormalize(x, m, s):
    return (x * s).groupby('time.month') + m

train_dataset_cnn_X_norm = (
    (train_dataset_cnn_X.groupby('time.month')
     - train_dataset_cnn_X.groupby('time.month').mean(dim={'time','lat','lon'})
     ).groupby('time.month')
     / train_dataset_cnn_X.groupby('time.month').std(dim={'time','lat','lon'})
).fillna(0)

val_dataset_cnn_X_norm = (
    (val_dataset_cnn_X.groupby('time.month')
     - train_dataset_cnn_X.groupby('time.month').mean(dim={'time','lat','lon'})
     ).groupby('time.month')
     / train_dataset_cnn_X.groupby('time.month').std(dim={'time','lat','lon'})
).fillna(0)

test_dataset_cnn_X_norm = (
    (test_dataset_cnn_X.groupby('time.month')
     - train_dataset_cnn_X.groupby('time.month').mean(dim={'time','lat','lon'})
     ).groupby('time.month')
     / train_dataset_cnn_X.groupby('time.month').std(dim={'time','lat','lon'})
).fillna(0)

train_dataset_cnn_y_norm = (
    (train_dataset_cnn_y.groupby('time.month')
     - train_dataset_cnn_y.groupby('time.month').mean(dim={'time','lat','lon'})
     ).groupby('time.month')
     / train_dataset_cnn_y.groupby('time.month').std(dim={'time','lat','lon'})
).fillna(0)

val_dataset_cnn_y_norm = (
    (val_dataset_cnn_y.groupby('time.month')
     - train_dataset_cnn_y.groupby('time.month').mean(dim={'time','lat','lon'})
     ).groupby('time.month')
     / train_dataset_cnn_y.groupby('time.month').std(dim={'time','lat','lon'})
).fillna(0)

test_dataset_cnn_y_norm = (
    (test_dataset_cnn_y.groupby('time.month')
     - train_dataset_cnn_y.groupby('time.month').mean(dim={'time','lat','lon'})
     ).groupby('time.month')
     / train_dataset_cnn_y.groupby('time.month').std(dim={'time','lat','lon'})
).fillna(0)

# Convert to numpy arrays and move axis
tr_X = np.moveaxis(np.array(train_dataset_cnn_X_norm.to_array()), 0, -1)
val_X = np.moveaxis(np.array(val_dataset_cnn_X_norm.to_array()), 0, -1)
test_X = np.moveaxis(np.array(test_dataset_cnn_X_norm.to_array()), 0, -1)
tr_y = np.moveaxis(np.array(train_dataset_cnn_y_norm.to_array()), 0, -1)
val_y = np.moveaxis(np.array(val_dataset_cnn_y_norm.to_array()), 0, -1)
test_y = np.moveaxis(np.array(test_dataset_cnn_y_norm.to_array()), 0, -1)

# Convert to PyTorch tensors
tr_X_tensor = torch.tensor(tr_X, dtype=torch.float32).to('cpu')
val_X_tensor = torch.tensor(val_X, dtype=torch.float32).to('cpu')
test_X_tensor = torch.tensor(test_X, dtype=torch.float32).to('cpu')
tr_y_tensor = torch.tensor(tr_y, dtype=torch.float32).to('cpu')
val_y_tensor = torch.tensor(val_y, dtype=torch.float32).to('cpu')
test_y_tensor = torch.tensor(test_y, dtype=torch.float32).to('cpu')

tr_X_tensor = tr_X_tensor.permute(0, 3, 1, 2)
val_X_tensor = val_X_tensor.permute(0, 3, 1, 2)
test_X_tensor = test_X_tensor.permute(0, 3, 1, 2)
tr_y_tensor = tr_y_tensor.permute(0, 3, 1, 2)
val_y_tensor = val_y_tensor.permute(0, 3, 1, 2)
test_y_tensor = test_y_tensor.permute(0, 3, 1, 2)

In [ ]:
def train(model, criterion, optimizer, train_inputs, train_targets, val_inputs, val_targets, batch_size=128, epochs=50, patience=5):
    model.train()
    dataset_size = len(train_inputs)
    val_size = len(val_inputs)

    best_val_loss = float('inf')
    patience_counter = 5

    for epoch in range(epochs):
        permutation = torch.randperm(dataset_size)
        for i in range(0, dataset_size, batch_size):
            indices = permutation[i:i + batch_size]
            batch_inputs, batch_targets = train_inputs[indices], train_targets[indices]

            optimizer.zero_grad()
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_targets)
            loss.backward()
            optimizer.step()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for i in range(0, val_size, batch_size):
                val_batch_inputs = val_inputs[i:i + batch_size]
                val_batch_targets = val_targets[i:i + batch_size]
                val_outputs = model(val_batch_inputs)
                val_loss += criterion(val_outputs, val_batch_targets).item()
        
        val_loss /= (val_size // batch_size)
        print(f'Epoch {epoch+1}/{epochs}, Training Loss: {loss.item()}, Validation Loss: {val_loss}')
              
train(model, criterion, optimizer, tr_X_tensor, tr_y_tensor, val_X_tensor, val_y_tensor)

In [ ]:
model.eval()

with torch.no_grad():
    predictions_norm = model(test_X_tensor)

# Convert predictions to numpy array
predictions_norm_np = predictions_norm.numpy()
predictions_norm_np = predictions_norm_np.squeeze(1)

# Define dimensions and coordinates (modify based on your data)
dims = ['time', 'lat', 'lon']
coords = {
    'time': test_dataset_cnn_y['time'].values,
    'lat': test_dataset_cnn_y['lat'].values,
    'lon': test_dataset_cnn_y['lon'].values
}

# Create an xarray DataArray
predictions_norm_da = xr.DataArray(predictions_norm_np, dims=dims, coords=coords)

In [ ]:
predictions_unnorm_da = (
    ( ( (predictions_norm_da-predictions_norm_da.mean(dim={'time','lat','lon'}))/predictions_norm_da.std(dim={'time','lat','lon'})).groupby('time.month')
     *train_dataset_cnn_y['ltg'].groupby('time.month').std(dim={'time','lat','lon'})
     ).groupby('time.month')
     +train_dataset_cnn_y['ltg'].groupby('time.month').mean(dim={'time','lat','lon'})
)

In [ ]:
predictions_unnorm_da = predictions_unnorm_da.where(predictions_unnorm_da >= 0, other = 0)

In [ ]:
predictions_unnorm_ds = predictions_unnorm_da.to_dataset(name='ltg')